In [9]:
#loading text
import requests

url = "https://raw.githubusercontent.com/humayra11/RAG-based-Chatbot-Company-Policies-/refs/heads/main/BDRCS%20HR%20POLICY.txt"

response = requests.get(url)
if response.status_code == 200:
    knowledge_text = response.text
    print("Text loaded successfully from GitHub.")
else:
    raise Exception(f"Failed to load  text. Status code: {response.status_code}")

#Chunking

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=20,
    length_function=len
)

chunks = text_splitter.split_text(knowledge_text)
print(f"Number of chunks {len(chunks)}.")

#Embedding

!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
chunk_embeddings = model.encode(chunks)

#Vector Store with FAISS

!pip install -q faiss-cpu
import faiss, numpy as np

d = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(np.array(chunk_embeddings).astype('float32'))

print(f"FAISS index created with {index.ntotal} vectors.")

#Load Generator Model
!pip install -q transformers
from transformers import pipeline
generator = pipeline('text2text-generation', model='google/flan-t5-small')

#Conversation History

conversation_history = []

#RAG Function with Memory and Citation
def ans_ques(query):

    query_embedding = model.encode([query]).astype('float32')
    k = 2
    distances, indices = index.search(query_embedding, k)
    retrieved_chunks = [chunks[i] for i in indices[0]]
    context = "\n\n".join(retrieved_chunks)

    past_context = "\n\n".join(
        [f"User: {h['query']}\nBot: {h['ans']}" for h in conversation_history[-3:]]
    )

    # Prompt for generation
    prompt_template = f"""
    Answer the question using only the context below.
    If the answer is not in the context, say "I don't have that information."

    Previous conversation:
    {past_context}

    Context:
    {context}

    ques:
    {query}

    ans:
    """

    # Genarate ans
    response = generator(prompt_template, max_length=150)[0]['generated_text']

    # Saving in conversation history
    conversation_history.append({"query": query, "ans": response})

    print(conversation_history)

    #citation
    citation_text = " (Source: github_src)"
    final_ans = f"{response}\n\n{citation_text}"

    return final_ans


#Gradio Interface

!pip install -q gradio
import gradio as gr

def chatbot_interface(query):
    ans = ans_ques(query)

    if "(Source:" in ans:
        main_ans, source = ans.split("(Source:", 1)
        source = "(Source:" + source
    else:
        main_ans, source = ans, "No source found"

    history_text = "\n\n".join([f"User: {h['query']}\nBot: {h['ans']}" for h in conversation_history])

    return main_ans.strip(), source.strip(), history_text

iface = gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(label="Ask about company policy"),
    outputs=[
        gr.Textbox(label="Chatbot ans"),
        gr.Textbox(label="Source"),
        gr.Textbox(label="Conversation History", lines=10)
    ],
    title="RAG-based Chatbot",
    description="Retrieves answers and shows conversation memory."
)
iface.launch(share=True)

Text loaded successfully from GitHub.
Number of chunks 170.
FAISS index created with 170 vectors.


Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c9c10430465bad262c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
